In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score
import pickle
from collections import defaultdict
from sklearn.utils import shuffle
import os
import cv2
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd

from torchinfo import summary
from sklearn.metrics import confusion_matrix
import seaborn as sns



In [2]:
import numpy as np
import os

# Define the folder path
folder_path = r"G:\CODING\py\data\numpy\mel\concatenatedspectrograms"

# Load the numpy files into the respective arrays with the correct capitalized naming
eeg_fold_1 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_1.npy'))
labels_fold_1 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_1.npy'))
patients_fold_1 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_1.npy'))

eeg_fold_2 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_2.npy'))
labels_fold_2 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_2.npy'))
patients_fold_2 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_2.npy'))

eeg_fold_3 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_3.npy'))
labels_fold_3 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_3.npy'))
patients_fold_3 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_3.npy'))

eeg_fold_4 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_4.npy'))
labels_fold_4 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_4.npy'))
patients_fold_4 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_4.npy'))

eeg_fold_5 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_5.npy'))
labels_fold_5 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_5.npy'))
patients_fold_5 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_5.npy'))

eeg_folds = [eeg_fold_1, eeg_fold_2, eeg_fold_3, eeg_fold_4, eeg_fold_5]
labels_folds = [labels_fold_1, labels_fold_2, labels_fold_3, labels_fold_4, labels_fold_5]
patients_folds = [patients_fold_1, patients_fold_2, patients_fold_3, patients_fold_4, patients_fold_5]

for i in range(len(eeg_folds)):
    eeg_folds[i] = eeg_folds[i].astype(np.float16)

In [3]:
def data_balancer(data, labels, factor):
    # Count the number of samples in each class
    num_class_0 = np.sum(labels == 0)
    num_class_1 = np.sum(labels == 1)
    num_class_2 = np.sum(labels == 2)

    # Find the minimum number of samples across all classes
    min_samples = min(num_class_0, num_class_1, num_class_2)

    # Calculate the number of samples to take from each class
    samples_per_class = min_samples // factor

    # Randomly sample 'samples_per_class' from each class
    class_0_indices = np.random.choice(np.where(labels == 0)[0], samples_per_class, replace=False)
    class_1_indices = np.random.choice(np.where(labels == 1)[0], samples_per_class, replace=False)
    class_2_indices = np.random.choice(np.where(labels == 2)[0], samples_per_class, replace=False)

    # Combine balanced indices
    balanced_indices = np.concatenate((class_0_indices, class_1_indices, class_2_indices))

    # Shuffle the balanced indices
    np.random.shuffle(balanced_indices)

    # Create balanced training data and labels
    balanced_data = data[balanced_indices]
    balanced_labels = labels[balanced_indices]

    return balanced_data, balanced_labels

In [4]:
class EarlyStopping:
    def __init__(self, patience=5):
        """
        Initializes the early stopping mechanism based on divergence detection.

        Args:
            patience (int): Number of consecutive epochs with increasing validation loss
                            before stopping.
        """
        self.patience = patience
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
        self.best_model_state = None

    def __call__(self, val_loss, model):
        """
        Checks if the validation loss is diverging and updates the state accordingly.

        Args:
            val_loss (float): Current epoch's validation loss.
            model (torch.nn.Module): The model being trained.
        """
        if self.best_loss is None or val_loss < self.best_loss:
            # Improvement detected
            self.best_loss = val_loss
            self.best_model_state = model.state_dict()
            self.counter = 0
        else:
            # Validation loss increased
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Divergence detected. Stopping training after {self.counter} epochs.")
                self.early_stop = True

    def load_best_model(self, model):
        """
        Restores the model to the state with the lowest validation loss.

        Args:
            model (torch.nn.Module): The model to restore.
        """
        model.load_state_dict(self.best_model_state)




In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class Inception(nn.Module):
    def __init__(self, fine_tune=True):
        super(Inception, self).__init__()
        

        inception = models.inception_v3(pretrained=True, aux_logits=True)
        del inception.AuxLogits 
        # inception.aux_logits = False
        # inception.AuxLogits = None

        old_conv1 = inception.Conv2d_1a_3x3.conv
        new_conv1 = nn.Conv2d(
            in_channels=3, 
            out_channels=old_conv1.out_channels, 
            kernel_size=old_conv1.kernel_size, 
            stride=old_conv1.stride, 
            padding=old_conv1.padding, 
            bias=old_conv1.bias is not None
        )

        old_conv2 = inception.Conv2d_2a_3x3.conv
        new_conv2 = nn.Conv2d(old_conv1.out_channels, old_conv2.out_channels, old_conv2.kernel_size, old_conv2.stride, old_conv2.padding, bias=old_conv2.bias is not None)

        old_conv3 = inception.Conv2d_2b_3x3.conv
        new_conv3 = nn.Conv2d(old_conv2.out_channels, old_conv3.out_channels, old_conv3.kernel_size, old_conv3.stride, old_conv3.padding, bias=old_conv3.bias is not None)


        # Copy weights for first 3 channels
        with torch.no_grad():
            new_conv1.weight[:, :3, :, :] = old_conv1.weight.clone()
            # new_conv1.weight[:, 3:, :, :] = torch.randn_like(new_conv1.weight[:, 3:, :, :]) * 0.01  
            nn.init.kaiming_normal_(new_conv1.weight[:, 3:, :, :], mode='fan_out', nonlinearity='relu')
            
            if old_conv1.bias is not None:
                new_conv1.bias.copy_(old_conv1.bias)

        # Replace layers in the model
        inception.Conv2d_1a_3x3.conv = new_conv1
        inception.Conv2d_2a_3x3.conv = new_conv2
        inception.Conv2d_2b_3x3.conv = new_conv3

        # self.feature_extractor = nn.Sequential(*list(inception.children())[:-1]) 
        self.feature_extractor = nn.Sequential(*list(inception.children())[:-2])

        # Adaptive pooling to ensure fixed-size output
        self.pooling = nn.AdaptiveAvgPool2d((1, 1))  # Converts (batch, 2048, 8, 8) → (batch, 2048, 1, 1)

        # Freeze all layers initially
        for param in inception.parameters():
            param.requires_grad = False  

        # Fine-tune specific layers
        if fine_tune:
            for param in inception.Mixed_7c.parameters():
                param.requires_grad = True 
            for param in inception.Mixed_7b.parameters():
                param.requires_grad = True  

        for module in self.feature_extractor.modules():
            if isinstance(module, nn.BatchNorm2d):
                module.eval()  # Fix batch statistics
                for param in module.parameters():
                    param.requires_grad = False

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.pooling(x)  # Make sure output is (batch_size, 2048, 1, 1)
        x = torch.flatten(x, start_dim=1)  # Flatten to (batch_size, 2048)
        return x

# Create model instance
modified_model = Inception(fine_tune=True)

class TransformerEncoder(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.1):
        super(TransformerEncoder, self).__init__()
        self.att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim)
        )
        self.layernorm1 = nn.LayerNorm(embed_dim)
        self.layernorm2 = nn.LayerNorm(embed_dim)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.dropout2 = nn.Dropout(dropout_rate)

    def forward(self, x):
        attn_output, _ = self.att(x, x, x)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)
    
class InceptionTransformerModel(nn.Module):
    def __init__(self, num_classes, embed_dim=2048, num_heads=8, ff_dim=4096, dropout_rate=0.1):
        super(InceptionTransformerModel, self).__init__()

        # Inception feature extractor
        self.feature_extractor = Inception()

        # Transformer Encoder
        self.transformer_encoder = TransformerEncoder(embed_dim, num_heads, ff_dim, dropout_rate)

        # Classification head
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = self.feature_extractor(x)  # (batch, 2048)
        x = x.unsqueeze(0)  # Convert to sequence format for Transformer (seq_len=1, batch, embed_dim)
        x = self.transformer_encoder(x)  # Apply Transformer Encoder
        x = x.squeeze(0)  # Back to (batch, embed_dim)
        return self.classifier(x)  # Final classification layer

num_classes = 3  
model = InceptionTransformerModel(num_classes)

# Print model structure
print(model)

c:\Users\Gabriel\anaconda3\envs\cudaenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Gabriel\anaconda3\envs\cudaenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


InceptionTransformerModel(
  (feature_extractor): Inception(
    (feature_extractor): Sequential(
      (0): BasicConv2d(
        (conv): Conv2d(20, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicConv2d(
        (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): BasicConv2d(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
      (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): BasicConv2d(
        (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stat

In [2]:
from torchinfo import summary
model = InceptionTransformerModel(num_classes)
summary(model)

Layer (type:depth-idx)                                  Param #
InceptionTransformerModel                               --
├─Inception: 1-1                                        --
│    └─Sequential: 2-1                                  --
│    │    └─BasicConv2d: 3-1                            (5,824)
│    │    └─BasicConv2d: 3-2                            (9,280)
│    │    └─BasicConv2d: 3-3                            (18,560)
│    │    └─MaxPool2d: 3-4                              --
│    │    └─BasicConv2d: 3-5                            (5,280)
│    │    └─BasicConv2d: 3-6                            (138,624)
│    │    └─MaxPool2d: 3-7                              --
│    │    └─InceptionA: 3-8                             (255,904)
│    │    └─InceptionA: 3-9                             (277,472)
│    │    └─InceptionA: 3-10                            (285,152)
│    │    └─InceptionB: 3-11                            (1,153,280)
│    │    └─InceptionC: 3-12                        

In [3]:
# Create a sample input tensor with shape [batch_size, channels, height, width]
batch_size = 4
sample_input = torch.randn(batch_size, 3, 224, 224)  # Using 3 channels temporarily
print(f"Input shape: {sample_input.shape}")

# Adjust channels for the model if needed (model expects 20 channels)
if model.feature_extractor.feature_extractor[0].conv.in_channels == 20:
    # Pad the input to have 20 channels
    expanded_input = torch.zeros(batch_size, 20, 224, 224)
    expanded_input[:, :3, :, :] = sample_input
    sample_input = expanded_input
    print(f"Adjusted input shape: {sample_input.shape}")

# Move the model and input to the same device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
sample_input = sample_input.to(device)

# Forward pass with gradient tracking disabled
with torch.no_grad():
    # Check feature extractor output
    feature_output = model.feature_extractor(sample_input)
    print(f"Feature extractor output shape: {feature_output.shape}")
    
    # Check transformer input/output (adding sequence dimension)
    transformer_input = feature_output.unsqueeze(0)
    print(f"Transformer input shape: {transformer_input.shape}")
    
    transformer_output = model.transformer_encoder(transformer_input)
    print(f"Transformer output shape: {transformer_output.shape}")
    
    # Final classification output
    classifier_input = transformer_output.squeeze(0)
    final_output = model.classifier(classifier_input)
    print(f"Final output shape: {final_output.shape}")
    
    # Complete forward pass
    complete_output = model(sample_input)
    print(f"Model output shape: {complete_output.shape}")
    
    # Check predicted classes
    _, predicted_classes = torch.max(complete_output, 1)
    print(f"Predicted classes: {predicted_classes}")

Input shape: torch.Size([4, 3, 224, 224])
Adjusted input shape: torch.Size([4, 20, 224, 224])
Feature extractor output shape: torch.Size([4, 2048])
Transformer input shape: torch.Size([1, 4, 2048])
Transformer output shape: torch.Size([1, 4, 2048])
Final output shape: torch.Size([4, 3])
Model output shape: torch.Size([4, 3])
Predicted classes: tensor([2, 2, 2, 2], device='cuda:0')


In [7]:
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# pca = sklearn.decomposition.PCA(3)
epochs = 100


fold_indices = np.arange(5)
fold_indices = np.random.permutation(fold_indices)
val_fold_indices = np.roll(fold_indices, 1)

input_shape = (3,224,224)
num_classes = 3


In [8]:
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# pca = sklearn.decomposition.PCA(3)
epochs = 100


fold_indices = np.arange(5)
fold_indices = np.random.permutation(fold_indices)
val_fold_indices = np.roll(fold_indices, 1)

input_shape = (3,224,224)
num_classes = 3


In [ ]:
import optuna
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import balanced_accuracy_score

def objective(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Hyperparameter suggestions
    num_heads = trial.suggest_categorical("num_heads", [2, 4, 8, 16, 32])
    droptout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    learning_rate = trial.suggest_float("learning_rate", 1e-7, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW", "SGD"])
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [10, 16, 32])
    label_smoothing = 0.3

    # Model
    foldmodel = InceptionTransformerModel(num_classes, embed_dim=2048, num_heads=num_heads, ff_dim=4096, dropout_rate=droptout_rate).to(device)

    criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
    optimizer_cls = {"Adam": optim.Adam, "AdamW": optim.AdamW, "SGD": optim.SGD}
    optimizer = optimizer_cls[optimizer_name](foldmodel.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Train-validation split (use last fold as test)
    test_fold = fold_indices[4]
    train_folds = fold_indices[:4]

    train_data = np.concatenate([eeg_folds[j] for j in train_folds]).transpose(0, 3, 1, 2)
    train_labels = np.concatenate([labels_folds[j] for j in train_folds])

    val_data = eeg_folds[test_fold].transpose(0, 3, 1, 2)
    val_labels = labels_folds[test_fold]

    # Balance dataset
    balanced_train_data, balanced_train_labels = data_balancer(train_data, train_labels, factor=2)

    # Convert to PyTorch tensors (keep tensors on CPU to avoid slow batch transfers)
    train_dataset = TensorDataset(torch.tensor(balanced_train_data, dtype=torch.float32), 
                                  torch.tensor(balanced_train_labels, dtype=torch.long))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

    val_dataset = TensorDataset(torch.tensor(val_data, dtype=torch.float32), 
                                torch.tensor(val_labels, dtype=torch.long))
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    # Training loop with early stopping & pruning
    epochs = 50  # Reduce max epochs
    best_val_loss = float("inf")
    patience = 10  # Early stopping patience
    patience_counter = 0

    for epoch in range(epochs):
        foldmodel.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad()
            outputs = foldmodel(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validation
        foldmodel.eval()
        val_loss, val_preds, val_labels_list = 0.0, [], []
        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_inputs, val_labels = val_inputs.to(device, non_blocking=True), val_labels.to(device, non_blocking=True)
                val_outputs = foldmodel(val_inputs)
                loss = criterion(val_outputs, val_labels)
                val_loss += loss.item()
                _, preds = torch.max(val_outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels_list.extend(val_labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_acc = balanced_accuracy_score(val_labels_list, val_preds)

        # Optuna Pruning
        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # Final evaluation
    foldmodel.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = foldmodel(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    test_acc = balanced_accuracy_score(all_labels, all_preds)
    print(f"Trial {trial.number}: Test Accuracy = {test_acc:.4f}")
    
    return test_acc

# Run Optuna optimization with parallel trials
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study.optimize(objective, n_trials=300, n_jobs=3)  # Reduce trials & use parallel execution

# Best parameters
print("Best hyperparameters:", study.best_params)
